In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
from random import uniform

# WebDriver 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # 브라우저를 화면에 띄우지 않고 실행하는 'Headless 모드'
chrome_options.add_argument('--no-sandbox')  # 샌드박스 모드 비활성화
chrome_options.add_argument('--disable-dev-shm-usage')  # /dev/shm 사용 비활성화
driver = webdriver.Chrome(options=chrome_options)  # 설정을 적용한 Chrome WebDriver 생성

# 키워드를 URL 인코딩 방식으로 변환하는 함수
def get_keyword(text):
    return text.replace(" ", "%20")

# 정렬 기준을 설정하는 함수 (조회수, 날짜 등)
def sort_kind(index):
    if index == 1:
        return 'vcount'  # 조회수 기준
    elif index == 2:
        return 'date'  # 날짜 기준
    else:
        return 'none'  # 기본 값

keyword = '강아지'  # 크롤링할 키워드 설정
driver.get('https://kin.naver.com/search/list.nhn?query=' + get_keyword(keyword))  # 네이버 지식인 검색 URL에 키워드 적용
time.sleep(uniform(0.1, 1.0))  # 랜덤으로 대기 시간 설정 (봇 탐지를 피하기 위해)

f = '2024.10.08'  # 크롤링 시작 일자
t = '2024.10.10'  # 크롤링 종료 일자
period_txt = "&period=" + f + ".%7C" + t + "."  # 기간 설정을 위한 문자열
_sort_kind = sort_kind(2)  # 날짜 기준으로 정렬

# 마지막 페이지 번호를 찾는 함수
def find_last_page():
    try:
        # 페이지 소스에서 페이지 네비게이션을 찾아 마지막 페이지 번호를 추출
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        page_list = soup.find('div', class_='paging')  # 페이징 영역 찾기
        last_page = page_list.find_all('a')[-1].get('href').split('page=')[-1]  # 마지막 페이지 번호 추출
        return int(last_page)
    except Exception as e:
        print("마지막 페이지를 찾는 중 오류 발생:", e)
        return 1  # 오류가 발생하면 기본 값으로 1 페이지만 크롤링

# 최대 페이지 수 설정 (실제 마지막 페이지 번호를 동적으로 추출)
max_pages = find_last_page()

page_index = 1  # 페이지 인덱스 초기화
page_url = []  # 페이지 URL을 저장할 리스트

# 페이지 단위로 크롤링 진행
while page_index <= max_pages:
    time.sleep(uniform(0.1, 1.0))  # 랜덤 대기 시간 추가
    # 검색 결과 페이지로 이동
    driver.get('https://kin.naver.com/search/list.nhn?' + "&sort=" + _sort_kind + '&query=' + get_keyword(keyword) + period_txt + "&section=kin" + "&page=" + str(page_index))
    html = driver.page_source  # 페이지 소스를 가져옴
    soup = BeautifulSoup(html, 'html.parser')

    # 각 질문의 URL을 가져옴
    tags = soup.find_all('a', class_="_searchListTitleAnchor")
    for tag in tags:
        url = tag['href']  # 질문 링크 추출
        page_url.append(url)

    page_index += 1  # 다음 페이지로 이동

# 텍스트 데이터 정리 함수 - 불필요한 공백 및 특정 키워드 제거
def clean_text(text):
    cleaned = text.replace('\n', ' ').replace('질문', '').replace('작성일', '').strip()  # \n, '질문', '작성일' 제거
    return cleaned

data = []  # 데이터를 저장할 리스트

# 각 질문 페이지를 순회하며 데이터 추출
for url in page_url:
    driver.get(url)  # 각 질문 페이지로 이동
    time.sleep(uniform(0.5, 1.0))  # 페이지 로딩 대기

    try:
        # 질문 제목 가져오기 (XPath 사용)
        title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[1]'))
        ).text
        title = clean_text(title)  # 제목 정리
    except:
        title = ""  # 제목이 없는 경우

    # 키워드가 제목에 포함되어 있는지 확인
    if keyword in title:
        try:
            # 질문 본문 가져오기 (XPath 사용)
            question_txt = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[3]'))
            ).text
            question_txt = clean_text(question_txt)  # 질문 본문 정리
        except:
            question_txt = ""  # 질문 본문이 없는 경우
        
        # 답변들 추출 (CSS Selector 사용)
        answers = driver.find_elements(By.CSS_SELECTOR, ".se-main-container .se-component-content")
        answer_text = [clean_text(answer.text) for answer in answers]  # 각 답변을 정리
        if not answer_text:
            answer_text = [""]  # 답변이 없는 경우 빈 문자열 추가

        # 작성일 가져오기 (XPath 사용)
        try:
            date = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[2]/span[3]'))
            ).text
            date = clean_text(date)  # 작성일 정리
        except:
            date = "날짜 없음"  # 작성일이 없는 경우

        # 전문의 여부 확인 (전문의 배지)
        try:
            expert_badge = driver.find_element(By.CSS_SELECTOR, '#answer_2 > div.profile_card._profileCardArea > a > div.card_info > div.profile_info > div.badge_area > span.badge.expert_job')
            expert_type = clean_text(expert_badge.text)  # 전문의 배지 텍스트 추출 및 정리
        except:
            expert_type = "일반 사용자"  # 전문의 배지가 없는 경우 일반 사용자로 표시

        # 채택 여부 확인 (채택 배지 유무)
        try:
            adopted_badge = driver.find_element(By.XPATH, '//*[@id="answer_2"]/div[3]/div/ul/li/strong')
            adopted_status = "채택"  # 채택된 답변
        except:
            adopted_status = "미채택"  # 채택되지 않은 답변

        # 질문 링크 저장
        question_link = driver.current_url

        # 모든 데이터를 리스트에 추가 (전문의 여부, 채택 여부 포함)
        for answer in answer_text:
            data.append([title, question_txt, answer, date, expert_type, adopted_status, question_link])

# 수집된 데이터를 데이터프레임으로 변환
dog_df = pd.DataFrame(data, columns=['제목', '질문', '답변', '날짜', '전문의 여부', '채택 여부', '질문 링크'])

# 데이터 출력
print(dog_df)

# 필요에 따라 데이터를 엑셀 파일로 저장 가능
# dog_df.to_excel('naver_kin_crawling_result.xlsx', index=False)

driver.quit()


마지막 페이지를 찾는 중 오류 발생: 'NoneType' object has no attribute 'find_all'
                            제목  \
0          강아지해외로 비행기 타고 데려갈경우   
1          강아지해외로 비행기 타고 데려갈경우   
2          강아지해외로 비행기 타고 데려갈경우   
3          강아지해외로 비행기 타고 데려갈경우   
4                   강아지 소변색...   
5                   강아지 소변색...   
6                  새끼 강아지 입질..   
7                  새끼 강아지 입질..   
8                  새끼 강아지 입질..   
9                  새끼 강아지 입질..   
10                 새끼 강아지 입질..   
11                 새끼 강아지 입질..   
12               산책을 무서워하는 강아지   
13               산책을 무서워하는 강아지   
14               산책을 무서워하는 강아지   
15               산책을 무서워하는 강아지   
16               산책을 무서워하는 강아지   
17               산책을 무서워하는 강아지   
18               산책을 무서워하는 강아지   
19                  1살강아지 스케일링   
20                  1살강아지 스케일링   
21  강아지 앞 발 마비 전신 마비 병명 좀 요 ㅠㅠ   
22  강아지 앞 발 마비 전신 마비 병명 좀 요 ㅠㅠ   
23                   강아지 나이...   

                                                   질문  \
0   키우는 강아지 해외로 데려가려면 어떤절